In [1]:
import numpy as np
from torchvision import datasets
import torch
import torch.nn as nn
from torch import optim 
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as transforms


In [2]:
torch.manual_seed(123)

In [3]:
transform = transforms.ToTensor()

In [4]:
train = datasets.CIFAR10(root = '.', train = True, download=True, transform = transform)
test = datasets.CIFAR10(root = '.', train = False, download=True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
train.data.shape

(50000, 32, 32, 3)

In [21]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128)
test_loader = torch.utils.data.DataLoader(test, batch_size=128)

In [22]:
len(test_loader)

79

In [23]:
train.data[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [40]:
class Classificador(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels=32, kernel_size= (3,3))
        self.conv2 = nn.Conv2d(32, 32, (3,3))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(num_features=32)
        self.pool = nn.MaxPool2d(kernel_size = (2,2))
        self.flatten = nn.Flatten()

        # output = (input - filter + 1) / stride
        # convolução 1 : (32 - 3 + 1) / 1 (padrão) = 30x30
        # pooling 1: 15x15
        # convolução 2: (15 - 3 + 1) / 1 = 13
        # pooling 2: 6x6
        # 6 x 6 x 32 (32: mapa de características)
        self.linear1 = nn.Linear(in_features=32*6*6, out_features=128) 
        self.linear2 = nn.Linear(128,128)
        self.output = nn.Linear(128,10)
        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, X):
        X = self.pool(self.bnorm(self.activation(self.conv1(X))))
        X = self.pool(self.bnorm(self.activation(self.conv2(X))))
        X = self.flatten(X)

        X = self.dropout(self.activation(self.linear1(X)))
        X = self.dropout(self.activation(self.linear2(X)))
        X = self.output(X)

        return X


In [41]:
net = Classificador()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [42]:
net

Classificador(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (bnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1152, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [44]:
net.to(device)

Classificador(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (bnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1152, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [45]:
def training_loop(loader, epoch):
    running_loss = 0.
    running_accuracy = 0.
    
    for i, data in enumerate(loader):
        inputs, labels = data
                
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()        
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()

        ps = F.softmax(outputs)
        top_p, top_class = ps.topk(k = 1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)

        accuracy = torch.mean(equals.type(torch.float))

        running_accuracy += accuracy

              
        # Imprimindo os dados referentes a este loop
        print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:03.2f} - precisão {:03.2f}'.format(epoch + 1, i + 1, len(loader), loss, 
                                   accuracy), end = '\r')
        
    # Imprimindo os dados referentes a esta época
    print('\rÉPOCA {:3d} FINALIZADA: perda {:.5f} - precisão {:.5f}'.format(epoch+1, running_loss/len(loader), 
                     running_accuracy/len(loader)))

In [46]:
for epoch in range(5):
    print('Treinando...')
    training_loop(train_loader, epoch) # Treina
    net.eval() # Modo de teste
    print('Validando...')
    training_loop(test_loader, epoch) # Testa
    net.train() # Modo de treino

Treinando...


C:\Users\vitao\AppData\Local\Temp\ipykernel_5372\2702599588.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ps = F.softmax(outputs)


ÉPOCA   1 FINALIZADA: perda 1.34219 - precisão 0.517175
Validando...
ÉPOCA   1 FINALIZADA: perda 1.40403 - precisão 0.500699
Treinando...
ÉPOCA   2 FINALIZADA: perda 1.01853 - precisão 0.641309
Validando...
ÉPOCA   2 FINALIZADA: perda 1.27983 - precisão 0.547072
Treinando...
ÉPOCA   3 FINALIZADA: perda 0.89848 - precisão 0.684322
Validando...
ÉPOCA   3 FINALIZADA: perda 1.13256 - precisão 0.599789
Treinando...
ÉPOCA   4 FINALIZADA: perda 0.81306 - precisão 0.715464
Validando...
ÉPOCA   4 FINALIZADA: perda 1.03056 - precisão 0.632221
Treinando...
ÉPOCA   5 FINALIZADA: perda 0.74813 - precisão 0.736767
Validando...
ÉPOCA   5 FINALIZADA: perda 0.95837 - precisão 0.663078
